In [1]:
!pip install -U bertopic
!pip install -U safetensors

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/dill-0.3.9-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_thunder-0.2.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/py

In [2]:
from bertopic import BERTopic
topic_model = BERTopic.load("MaartenGr/BERTopic_Wikipedia")

topic_model.get_topic_info()

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Topic,Count,Name,Representation,Representative_Docs
0,-1,633881,-1_cast_films_film_movie,"[cast, films, film, movie, 2020, comedy, relea...",NaN
1,0,18441,0_goalscorer_scored_goals_goal,"[goalscorer, scored, goals, goal, goalkeeper, ...",NaN
2,1,8518,1_khan_actor_raj_shah,"[khan, actor, raj, shah, crore, hai, actress, ...",NaN
3,2,7521,2_married_divorced_couple_remarried,"[married, divorced, couple, remarried, engaged...",NaN
4,3,6765,3_cast_actress_starred_actor,"[cast, actress, starred, actor, actors, starri...",NaN
...,...,...,...,...,...
2372,2371,30,2371_paintings_painting_paint_art,"[paintings, painting, paint, art, artist, gall...",NaN
2373,2372,30,2372_tulips_tulip_economists_economic,"[tulips, tulip, economists, economic, bulbs, 1...",NaN
2374,2373,30,2373_squads_squad_roster_players,"[squads, squad, roster, players, teams, tourna...",NaN
2375,2374,30,2374_entrances_subterranean_tunnel_stairs,"[entrances, subterranean, tunnel, stairs, pyra...",NaN


In [3]:
new_text = ["This document is about deep learning and neural networks."]

topics, probs = topic_model.transform(new_text)

print("Predicted topic:", topics[0])
print("Probability:", probs[0])
print("Representative words:", topic_model.get_topic(topics[0]))
topic_model.get_topic_info(topics[0])

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.98s/it]
2025-07-02 15:02:01,198 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


Predicted topic: 181
Probability: 0.62471
Representative words: [['neural', 0.5749497413635254], ['neuron', 0.5047926306724548], ['neurons', 0.4919857978820801], ['convolutions', 0.48135024309158325], ['backpropagation', 0.4702264666557312], ['layers', 0.45920509099960327], ['convolutional', 0.45698797702789307], ['networks', 0.44069626927375793], ['recognition', 0.42982780933380127], ['convolution', 0.4223356544971466]]


,Topic,Count,Name,Representation,Representative_Docs
0,181,298,181_neural_neuron_neurons_convolutions,"[neural, neuron, neurons, convolutions, backpr...",NaN


In [4]:
embedding_model = topic_model.embedding_model

In [5]:
import torch

# Check if embedding model is using GPU
print("CUDA available:", torch.cuda.is_available())

# Send the model to GPU and check its device
if hasattr(embedding_model, 'embedding_model'):  # For sentence-transformers
    print("Embedding model device:", next(embedding_model.embedding_model.parameters()).device)

CUDA available: True
Embedding model device: cuda:0


In [6]:
import pandas as pd

df = pd.read_csv(
    "data/bcom_reviews.csv",
    engine="python",
)

df.loc[0:10]

,id,source,headline,negative,positive
0,5548,original,Beautiful stay,I got quite cold at night because the heating ...,"Spacious, clean and comfortable apartment. Gre..."
1,5547,original,great potential to be at an advanced level,This place has great potential to be at an adv...,"In this accommodation, we had exactly what we ..."
2,5541,original,NaN,-Building itself needs work on the inside but ...,clean bright apartment excellent location well...
3,5540,original,"Our stay was comfortable, but unfortunately, t...",Check in procedure,"Clean, safe, facilities"
4,5538,original,NaN,some people were too loudly,NaN
5,5534,original,Great,NaN,Clean and spacious. Kitchen was provided what ...
6,5533,original,Great location for Congress,Construction on building exterior started at 7...,Location is great for the Congress Center.
7,5524,original,"I would to come again, and again, and if it's ...",NaN,"Greetings from Serbia, it was phenomenal 2 nig..."
8,5521,original,NaN,NaN,The apartment is modern and clean. Everything ...
9,5520,original,"Great place , super clean and easy access. We ...",The noise from the street,Via booking


In [7]:
len(df)

1116

In [8]:
from IPython.display import display

new_text = df.loc[0:10, "positive"].tolist()
topics, probs = topic_model.transform(new_text)

for i in range(len(topics)):
    print("------------------------------")
    print(new_text[i])
    print("------------------------------")
    display(topic_model.get_topic_info(topics[i]))

Batches: 100%|██████████| 1/1 [00:00<00:00, 37.71it/s]
2025-07-02 15:03:18,793 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


------------------------------
Spacious, clean and comfortable apartment. Great amenities including body lotion, toothbrush kit etc. Kitchen was easy to cook, located in a great area with grocers, shops, and cafes nearby. The staff online were very helpful - I had an issue with the electronic key ad and they responded promptly even during the evening, and they even sent a technician to fix the door in less than 30 minutes.
------------------------------


,Topic,Count,Name,Representation,Representative_Docs
0,1136,65,1136_restaurants_restaurant_chefs_culinary,"[restaurants, restaurant, chefs, culinary, cui...",NaN


------------------------------
In this accommodation, we had exactly what we needed, comfort "like at home". When you have a two-year-old child, you have to prepare meals for your family on a trip. The building provided us with everything we needed from a judge, a washing machine, an oven, a stove, an iron for clothes, comfort.
------------------------------


,Topic,Count,Name,Representation,Representative_Docs
0,1502,51,1502_rooms_room_palace_furnishings,"[rooms, room, palace, furnishings, ballroom, p...",NaN


...

,Topic,Count,Name,Representation,Representative_Docs
0,622,101,622_gentrification_suburbanization_gentrified_...,"[gentrification, suburbanization, gentrified, ...",NaN


------------------------------
Via booking
------------------------------


,Topic,Count,Name,Representation,Representative_Docs
0,840,82,840_airline_airlines_seats_seating,"[airline, airlines, seats, seating, 737, aisle...",NaN


------------------------------
Great location and easy walking to city highlights. The apartment was well equipped and comfortable. The hosts were very responsive and helpful.
------------------------------


,Topic,Count,Name,Representation,Representative_Docs
0,563,110,563_piazza_boulevard_della_buildings,"[piazza, boulevard, della, buildings, baroque,...",NaN


In [82]:
from sentence_transformers import SentenceTransformer

def clean_text(text):
    return text.replace('\r', ' ').replace('\n', ' ').strip()

docs = [clean_text(t) for t in df.loc[:, "headline"].dropna()] + [clean_text(t) for t in df.loc[:, "positive"].dropna()] + [clean_text(t) for t in df.loc[:, "negative"].dropna()]

# Create embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(docs, show_progress_bar=True)

Batches: 100%|██████████| 79/79 [00:00<00:00, 172.54it/s]


In [86]:
for text in docs:
    if text.strip() == "":
        print("HIT")

In [78]:
import collections
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

# Extract vocab to be used in BERTopic
vocab = collections.Counter()
tokenizer = CountVectorizer().build_tokenizer()
for doc in tqdm(docs):
  vocab.update(tokenizer(doc))
vocab = [word for word, frequency in vocab.items() if frequency >= 5]; len(vocab)

100%|██████████| 2501/2501 [00:00<00:00, 106505.65it/s]


1348

In [79]:
from cuml.manifold import UMAP
from cuml.cluster import HDBSCAN

# Prepare sub-models
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
umap_model = UMAP(n_components=5, n_neighbors=50, random_state=42, metric="cosine", verbose=True)
hdbscan_model = HDBSCAN(min_samples=20, gen_min_span_tree=True, prediction_data=False, min_cluster_size=20, verbose=True)
vectorizer_model = CountVectorizer(vocabulary=vocab, stop_words="english")

# Fit BERTopic without actually performing any clustering
topic_model= BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer_model,
        verbose=True
).fit(docs, embeddings=embeddings)

2025-06-05 16:35:52,972 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-05 16:35:53,117 - BERTopic - Dimensionality - Completed ✓
2025-06-05 16:35:53,118 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-05 16:35:53,175 - BERTopic - Cluster - Completed ✓
2025-06-05 16:35:53,177 - BERTopic - Representation - Fine-tuning topics using representation models.
/storage/brno2/home/povolji2/.local/lib/python3.12/site-packages/bertopic/vectorizers/_ctfidf.py:82: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df) + 1)
2025-06-05 16:35:53,207 - BERTopic - Representation - Completed ✓


[D] [16:35:52.974055] /home/coder/cuml/cpp/src/umap/runner.cuh:107 n_neighbors=50
[D] [16:35:52.974114] /home/coder/cuml/cpp/src/umap/runner.cuh:129 Calling knn graph run
[D] [16:35:52.975196] /home/coder/cuml/cpp/src/umap/runner.cuh:135 Done. Calling fuzzy simplicial set
[D] [16:35:52.975648] /home/coder/cuml/cpp/src/umap/fuzzy_simpl_set/naive.cuh:318 Smooth kNN Distances
[D] [16:35:52.975683] /home/coder/cuml/cpp/src/umap/fuzzy_simpl_set/naive.cuh:320 sigmas = [ 0.0916085, 0.32506, 0.207621, 0.11274, 0.124256, 0.275358, 0.0184921, 0.0231256, 0.116459, 0.147078, 0.158334, 0.208957, 0.0782707, 0.0842767, 0.0282083, 0.0264292, 0.268708, 0.0413684, 0.0146273, 0.0534956, 0.0617596, 0.0916054, 0.0425339, 0.0130491, 0.0206522 ]

[D] [16:35:52.975708] /home/coder/cuml/cpp/src/umap/fuzzy_simpl_set/naive.cuh:322 rhos = [ 0.148804, 3.57628e-07, 1.78814e-07, 1.19209e-07, 0.110781, 2.98023e-07, 0.291587, 0.284363, 1.78814e-07, 2.98023e-07, 3.57628e-07, 2.38419e-07, 2.38419e-07, 5.96046e-08, 0.224

In [80]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,71,-1_bit_property_garden_think,"[bit, property, garden, think, baby, ve, good,...","[Everything went smooth, the check-in and chec..."
1,0,2172,0_apartment_location_great_clean,"[apartment, location, great, clean, good, nice...","[Nice apartment in top location, Great locatio..."
2,1,217,1_prague_apartment_stay_days,"[prague, apartment, stay, days, location, plac...","[Stay in Prague, Strongly endorsed if your iti..."
3,2,41,2_like_negative_12_fridge,"[like, negative, 12, fridge, rate, bottle, ter...","[There is nothing not to like!, Nothing much. ..."


In [81]:
topic_model.get_topic_info(0)

,Topic,Count,Name,Representation,Representative_Docs
0,0,2172,0_apartment_location_great_clean,"[apartment, location, great, clean, good, nice...","[Nice apartment in top location, Great locatio..."


In [ ]:
# Prepare sub-models
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
umap_model = UMAP(n_components=5, n_neighbors=15, random_state=42, metric="cosine", verbose=True)
hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
vectorizer_model = CountVectorizer(vocabulary=vocab, stop_words="english", min_df=2, ngram_range=(1, 2))

# Fit BERTopic without actually performing any clustering
topic_model= BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer_model,
        top_n_words=10,
        verbose=True
).fit(docs, embeddings=embeddings)